In [27]:
import backtrader as bt
import pandas as pd

data = bt.feeds.GenericCSVData(
    dataname="../../data/SOLUSDT_FIVE2025_15m.csv",
    dtformat='%Y-%m-%d %H:%M:%S',
    timeframe=bt.TimeFrame.Minutes,
    compression=15,
    datetime=0,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
    openinterest=-1  # No open interest in your CSV
)


class EmaCrossoverShort(bt.Strategy):
    params = (
        ('short_period', 9),
        ('long_period', 21),
        ('position_pct', 0.95),
        ('risk_pct', 0.5),     # 0.5% risk
        ('reward_pct', 1.0),   # 1% reward (1:2 risk:reward)
    )

    def __init__(self):
        self.short_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.short_period)
        self.long_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.long_period)

        self.crossover = bt.indicators.CrossOver(self.short_ema, self.long_ema)

          # Initialize DataFrame with proper dtypes
        self.trade_pairs = pd.DataFrame({
            'trade_number': pd.Series(dtype='int'),
            'entry_type': pd.Series(dtype='str'),
            'entry_price': pd.Series(dtype='float'),
            'entry_date': pd.Series(dtype='object'),  # For date only
            'entry_time': pd.Series(dtype='object'),  # For time only
            'exit_type': pd.Series(dtype='str'),
            'exit_price': pd.Series(dtype='float'),
            'exit_date': pd.Series(dtype='object'),   # For date only
            'exit_time': pd.Series(dtype='object'),   # For time onl
            'size': pd.Series(dtype='float'),
            'pnl': pd.Series(dtype='float'),
            'pnl_percentage': pd.Series(dtype='float') 
        })
        
        self.trade_counter = 0  # To track trade pair numbers
        self.current_trade = None  # To track current open trade

        # Simple counters
        self.total_trades = 0
        self.winning_trades = 0
        self.total_pnl = 0.0

        self.order= None  # To keep track of pending orders

        self.entry_price = None
        self.stop_loss_price = None
        self.take_profit_price = None
    
    
    def next(self):
        if len(self) < self.params.long_period:
            return
        
        # Skip if we have pending orders
        if self.order:
            return
        
        # Get current position
        position = self.position.size
        current_price = self.data.close[0]

        # Check for stop loss or take profit if we have a position
        if position < 0:  # We have a short position
            if current_price >= self.stop_loss_price:
                # Stop loss hit
                self.order = self.buy(size=abs(position))
                return
            elif current_price <= self.take_profit_price:
                # Take profit hit
                self.order = self.buy(size=abs(position))
                return

        if self.crossover < 0 and position == 0:
            # BEARISH CROSSOVER - Going SHORT
            cash = self.broker.getcash()
            price = self.data.close[0]
            size = (cash * self.params.position_pct) / price
            
            # Set entry price and calculate stop loss/take profit
            self.entry_price = price
            self.stop_loss_price = price * (1 + self.params.risk_pct / 100)  # Price goes up = loss for short
            self.take_profit_price = price * (1 - self.params.reward_pct / 100)  # Price goes down = profit for short
            
            self.order = self.sell(size=size)
        
        elif self.crossover > 0 and position < 0:
            # BULLISH CROSSOVER - Closing SHORT (keep this as backup)
            self.order = self.buy(size=abs(position))


    def stop(self):
        position = self.position.size
        
        if position != 0:
            print(f"🔚 END OF DATA - Force closing position: {position} units")
            
            # If we have an open trade, close it
            if self.current_trade:
                current_datetime = self.data.datetime.datetime(0)
                current_price = self.data.close[0]
                
                if position > 0:
                    # Closing LONG position
                    self.current_trade['exit_type'] = 'SELL'
                    self.current_trade['exit_price'] = current_price
                    self.current_trade['exit_date'] = current_datetime.date()
                    self.current_trade['exit_time'] = current_datetime.time()

                    
                    # Calculate PnL for LONG trade
                    pnl = (current_price - self.current_trade['entry_price']) * self.current_trade['size']
                    self.current_trade['pnl'] = pnl

                    # pnl = (current_price - self.current_trade['entry_price']) * self.current_trade['size']
                    # self.current_trade['pnl'] = pnl
                    pnl_percentage = ((current_price - self.current_trade['entry_price']) / self.current_trade['entry_price']) * 100
                    self.current_trade['pnl_percentage'] = pnl_percentage
                    
                else:
                    # Closing SHORT position
                    self.current_trade['exit_type'] = 'BUY'
                    self.current_trade['exit_price'] = current_price
                    self.current_trade['exit_date'] = current_datetime.date()
                    self.current_trade['exit_time'] = current_datetime.time()
                    
                    # Calculate PnL for SHORT trade
                    pnl = (self.current_trade['entry_price'] - current_price) * self.current_trade['size']
                    self.current_trade['pnl'] = pnl
                    pnl_percentage = ((self.current_trade['entry_price'] - current_price) / self.current_trade['entry_price']) * 100
                    self.current_trade['pnl_percentage'] = pnl_percentage
                
                # Add the forced-closed trade to DataFrame
                self.trade_pairs = pd.concat([self.trade_pairs, pd.DataFrame([self.current_trade])], ignore_index=True)
                self.current_trade = None
            
            # Your existing closing logic...
            if position > 0:
                self.sell(size=abs(position))
            else:
                self.buy(size=abs(position))
            print("=" * 50)
   
    def notify_order(self, order):
        if order.status == order.Completed:
            current_datetime = self.data.datetime.datetime(0)
            
            if order.isbuy():
                # print(f"BUY EXECUTED at ${order.executed.price:.2f} , position size: {order.executed.size}")
                
                # If this is closing a SHORT position (buying to close)
                if self.current_trade and self.current_trade['entry_type'] == 'SELL':
                    # This is an EXIT - complete the trade pair
                    self.current_trade['exit_type'] = 'BUY'
                    self.current_trade['exit_price'] = order.executed.price
                    self.current_trade['exit_date'] = current_datetime.date()
                    self.current_trade['exit_time'] = current_datetime.time()
                    
                    # Calculate PnL for SHORT trade: (entry_price - exit_price) * size
                    pnl = (self.current_trade['entry_price'] - order.executed.price) * self.current_trade['size']
                    self.current_trade['pnl'] = pnl
                    pnl_percentage = ((self.current_trade['entry_price'] - order.executed.price) / self.current_trade['entry_price']) * 100
                    self.current_trade['pnl_percentage'] = pnl_percentage
                    
                    # Add completed trade to DataFrame
                    self.trade_pairs = pd.concat([self.trade_pairs, pd.DataFrame([self.current_trade])], ignore_index=True)
                    self.current_trade = None  # Reset current trade
                    
                else:
                    # This is an ENTRY for a LONG position
                    self.trade_counter += 1
                    self.current_trade = {
                        'trade_number': self.trade_counter,
                        'entry_type': 'BUY',
                        'entry_price': order.executed.price,
                        'entry_date': current_datetime.date(),
                        'entry_time': current_datetime.time(),
                        'exit_type': None,
                        'exit_price': None,
                        'exit_date': None,
                        'exit_time': None,
                        'size': abs(order.executed.size),
                        'pnl': None,
                        'pnl_percentage': None
                    }
                    
            elif order.issell():
                # print(f"SELL EXECUTED at ${order.executed.price:.2f} , position size: {order.executed.size}")
                
                # If this is closing a LONG position (selling to close)
                if self.current_trade and self.current_trade['entry_type'] == 'BUY':
                    # This is an EXIT - complete the trade pair
                    self.current_trade['exit_type'] = 'SELL'
                    self.current_trade['exit_price'] = order.executed.price
                    self.current_trade['exit_date'] = current_datetime.date()
                    self.current_trade['exit_time'] = current_datetime.time()
                    
                    # Calculate PnL for LONG trade: (exit_price - entry_price) * size
                    pnl = (order.executed.price - self.current_trade['entry_price']) * self.current_trade['size']
                    self.current_trade['pnl'] = pnl
                    pnl_percentage = ((order.executed.price - self.current_trade['entry_price']) / self.current_trade['entry_price']) * 100
                    self.current_trade['pnl_percentage'] = pnl_percentage


                    # Add completed trade to DataFrame
                    self.trade_pairs = pd.concat([self.trade_pairs, pd.DataFrame([self.current_trade])], ignore_index=True)
                    self.current_trade = None  # Reset current trade
                    
                else:
                    # This is an ENTRY for a SHORT position
                    self.trade_counter += 1
                    self.current_trade = {
                        'trade_number': self.trade_counter,
                        'entry_type': 'SELL',
                        'entry_price': order.executed.price,
                        'entry_date': current_datetime.date(),
                        'entry_time': current_datetime.time(),
                        'exit_type': None,
                        'exit_price': None,
                        'exit_date': None,
                        'exit_time': None,
                        'size': abs(order.executed.size),
                        'pnl': None,
                        'pnl_percentage': None
                    }
        
        # Your existing error handling code...
        elif order.status == order.Margin:
            print(f"   ❌ ORDER REJECTED - Insufficient Margin/Cash")
        elif order.status == order.Rejected:
            print(f"   ❌ ORDER REJECTED - {order.info}")
        elif order.status == order.Canceled:
            print(f"   ⚠️ ORDER CANCELED")
            
        # Reset self.order when order completes
        if order.status in [order.Completed, order.Canceled, order.Rejected, order.Margin]:
            self.order = None


    # def notify_trade(self, trade):
    #     """Handle trade notifications"""
    #     if trade.isclosed:
    #         self.total_trades += 1
    #         pnl = trade.pnl
    #         self.total_pnl += pnl

    #         # Determine trade direction
    #         direction = "LONG" if trade.size > 0 else "SHORT"

    #         if pnl > 0:
    #             self.winning_trades += 1
    #             print(f"   ✅ WINNING {direction} TRADE - P&L: ${pnl:.2f}")
    #         else:
    #             print(f"   ❌ LOSING {direction} TRADE - P&L: ${pnl:.2f}")

    #         # Print running statistics
    #         win_rate = (self.winning_trades / self.total_trades) * 100
    #         print(
    #             f"   📊 Total: {self.total_trades} | Win Rate: {win_rate:.1f}% | Total P&L: ${self.total_pnl:.2f}")
    #         print("=" * 50)




class EmaCrossoverLong(bt.Strategy):
    params = (
        ('short_period', 9),
        ('long_period', 21),
        ('position_pct', 0.95),
        ('risk_pct', 0.5),     # 0.5% risk
        ('reward_pct', 1),   # 1% reward (1:2 risk:reward)
    )

    def __init__(self):
        self.short_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.short_period)
        self.long_ema = bt.indicators.ExponentialMovingAverage(
            self.data.close, period=self.params.long_period)

        self.crossover = bt.indicators.CrossOver(self.short_ema, self.long_ema)

        # Initialize DataFrame with proper dtypes
        self.trade_pairs = pd.DataFrame({
            'trade_number': pd.Series(dtype='int'),
            'entry_type': pd.Series(dtype='str'),
            'entry_price': pd.Series(dtype='float'),
            'entry_date': pd.Series(dtype='object'),  # For date only
            'entry_time': pd.Series(dtype='object'),  # For time only
            'exit_type': pd.Series(dtype='str'),
            'exit_price': pd.Series(dtype='float'),
            'exit_date': pd.Series(dtype='object'),   # For date only
            'exit_time': pd.Series(dtype='object'),   # For time only
            'size': pd.Series(dtype='float'),
            'pnl': pd.Series(dtype='float'),
            'pnl_percentage': pd.Series(dtype='float') 
        })
        
        self.trade_counter = 0  # To track trade pair numbers
        self.current_trade = None  # To track current open trade

        # Simple counters
        self.total_trades = 0
        self.winning_trades = 0
        self.total_pnl = 0.0

        self.order= None  # To keep track of pending orders

        self.entry_price = None
        self.stop_loss_price = None
        self.take_profit_price = None

    def next(self):
        if len(self) < self.params.long_period:
            return
        
        # Skip if we have pending orders
        if self.order:
            return
        
        # Get current position
        position = self.position.size
        current_price = self.data.close[0]

        # Check for stop loss or take profit if we have a position
        if position > 0:  # We have a long position
            if current_price <= self.stop_loss_price:
                # Stop loss hit
                self.order = self.sell(size=abs(position))
                return
            elif current_price >= self.take_profit_price:
                # Take profit hit
                self.order = self.sell(size=abs(position))
                return

        if self.crossover > 0 and position == 0:
            # BULLISH CROSSOVER - Going LONG
            cash = self.broker.getcash()
            price = self.data.close[0]
            size = (cash * self.params.position_pct) / price
            
            # Set entry price and calculate stop loss/take profit
            self.entry_price = price
            self.stop_loss_price = price * (1 - self.params.risk_pct / 100)  # Price goes down = loss for long
            self.take_profit_price = price * (1 + self.params.reward_pct / 100)  # Price goes up = profit for long
            
            self.order = self.buy(size=size)

        elif self.crossover < 0 and position > 0:
            # BEARISH CROSSOVER - Closing LONG (keep this as backup)
            self.order = self.sell(size=abs(position))

    def stop(self):
        position = self.position.size
        
        if position != 0:
            print(f"🔚 END OF DATA - Force closing position: {position} units")
            
            # If we have an open trade, close it
            if self.current_trade:
                current_datetime = self.data.datetime.datetime(0)
                current_price = self.data.close[0]
                
                if position > 0:
                    # Closing LONG position
                    self.current_trade['exit_type'] = 'SELL'
                    self.current_trade['exit_price'] = current_price
                    self.current_trade['exit_date'] = current_datetime.date()
                    self.current_trade['exit_time'] = current_datetime.time()
                    
                    # Calculate PnL for LONG trade
                    pnl = (current_price - self.current_trade['entry_price']) * self.current_trade['size']
                    self.current_trade['pnl'] = pnl

                    # pnl = (current_price - self.current_trade['entry_price']) * self.current_trade['size']
                    # self.current_trade['pnl'] = pnl
                    pnl_percentage = ((current_price - self.current_trade['entry_price']) / self.current_trade['entry_price']) * 100
                    self.current_trade['pnl_percentage'] = pnl_percentage
                    
                else:
                    # Closing SHORT position
                    self.current_trade['exit_type'] = 'BUY'
                    self.current_trade['exit_price'] = current_price
                    self.current_trade['exit_date'] = current_datetime.date()
                    self.current_trade['exit_time'] = current_datetime.time()
                    
                    # Calculate PnL for SHORT trade
                    pnl = (self.current_trade['entry_price'] - current_price) * self.current_trade['size']
                    self.current_trade['pnl'] = pnl
                    pnl_percentage = ((self.current_trade['entry_price'] - current_price) / self.current_trade['entry_price']) * 100
                    self.current_trade['pnl_percentage'] = pnl_percentage
                
                # Add the forced-closed trade to DataFrame
                self.trade_pairs = pd.concat([self.trade_pairs, pd.DataFrame([self.current_trade])], ignore_index=True)
                self.current_trade = None
            
            # Your existing closing logic...
            if position > 0:
                self.sell(size=abs(position))
            else:
                self.buy(size=abs(position))
            print("=" * 50)
   
    def notify_order(self, order):
        if order.status == order.Completed:
            current_datetime = self.data.datetime.datetime(0)
            
            if order.isbuy():
                # print(f"BUY EXECUTED at ${order.executed.price:.2f} , position size: {order.executed.size}")
                
                # If this is closing a SHORT position (buying to close)
                if self.current_trade and self.current_trade['entry_type'] == 'SELL':
                    # This is an EXIT - complete the trade pair
                    self.current_trade['exit_type'] = 'BUY'
                    self.current_trade['exit_price'] = order.executed.price
                    self.current_trade['exit_date'] = current_datetime.date()
                    self.current_trade['exit_time'] = current_datetime.time()
                    
                    # Calculate PnL for SHORT trade: (entry_price - exit_price) * size
                    pnl = (self.current_trade['entry_price'] - order.executed.price) * self.current_trade['size']
                    self.current_trade['pnl'] = pnl
                    pnl_percentage = ((self.current_trade['entry_price'] - order.executed.price) / self.current_trade['entry_price']) * 100
                    self.current_trade['pnl_percentage'] = pnl_percentage
                    
                    # Add completed trade to DataFrame
                    self.trade_pairs = pd.concat([self.trade_pairs, pd.DataFrame([self.current_trade])], ignore_index=True)
                    self.current_trade = None  # Reset current trade
                    
                else:
                    # This is an ENTRY for a LONG position
                    self.trade_counter += 1
                    self.current_trade = {
                        'trade_number': self.trade_counter,
                        'entry_type': 'BUY',  # or 'SELL'
                        'entry_price': order.executed.price,
                        'entry_date': current_datetime.date(),
                        'entry_time': current_datetime.time(),
                        'exit_type': None,
                        'exit_price': None,
                        'exit_date': None,
                        'exit_time': None,
                        'size': abs(order.executed.size),
                        'pnl': None,
                        'pnl_percentage': None
                    }
                    
            elif order.issell():
                # print(f"SELL EXECUTED at ${order.executed.price:.2f} , position size: {order.executed.size}")
                
                # If this is closing a LONG position (selling to close)
                if self.current_trade and self.current_trade['entry_type'] == 'BUY':
                    # This is an EXIT - complete the trade pair
                    self.current_trade['exit_type'] = 'SELL'
                    self.current_trade['exit_price'] = order.executed.price
                    self.current_trade['exit_date'] = current_datetime.date()
                    self.current_trade['exit_time'] = current_datetime.time()
                    
                    # Calculate PnL for LONG trade: (exit_price - entry_price) * size
                    pnl = (order.executed.price - self.current_trade['entry_price']) * self.current_trade['size']
                    self.current_trade['pnl'] = pnl
                    pnl_percentage = ((order.executed.price - self.current_trade['entry_price']) / self.current_trade['entry_price']) * 100
                    self.current_trade['pnl_percentage'] = pnl_percentage


                    # Add completed trade to DataFrame
                    self.trade_pairs = pd.concat([self.trade_pairs, pd.DataFrame([self.current_trade])], ignore_index=True)
                    self.current_trade = None  # Reset current trade
                    
                else:
                    # This is an ENTRY for a SHORT position
                    self.trade_counter += 1
                    self.current_trade = {
                        'trade_number': self.trade_counter,
                        'entry_type': 'BUY',  # or 'SELL'
                        'entry_price': order.executed.price,
                        'entry_date': current_datetime.date(),
                        'entry_time': current_datetime.time(),
                        'exit_type': None,
                        'exit_price': None,
                        'exit_date': None,
                        'exit_time': None,
                        'size': abs(order.executed.size),
                        'pnl': None,
                        'pnl_percentage': None
                    }
        
        # Your existing error handling code...
        elif order.status == order.Margin:
            print(f"   ❌ ORDER REJECTED - Insufficient Margin/Cash")
        elif order.status == order.Rejected:
            print(f"   ❌ ORDER REJECTED - {order.info}")
        elif order.status == order.Canceled:
            print(f"   ⚠️ ORDER CANCELED")
            
        # Reset self.order when order completes
        if order.status in [order.Completed, order.Canceled, order.Rejected, order.Margin]:
            self.order = None


    # def notify_trade(self, trade):
    #     """Handle trade notifications"""
    #     if trade.isclosed:
    #         self.total_trades += 1
    #         pnl = trade.pnl
    #         self.total_pnl += pnl

    #         # Determine trade direction
    #         direction = "LONG" if trade.size > 0 else "SHORT"

    #         if pnl > 0:
    #             self.winning_trades += 1
    #             print(f"   ✅ WINNING {direction} TRADE - P&L: ${pnl:.2f}")
    #         else:
    #             print(f"   ❌ LOSING {direction} TRADE - P&L: ${pnl:.2f}")

    #         # Print running statistics
    #         win_rate = (self.winning_trades / self.total_trades) * 100
    #         print(
    #             f"   📊 Total: {self.total_trades} | Win Rate: {win_rate:.1f}% | Total P&L: ${self.total_pnl:.2f}")
    #         print("=" * 50)

  

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(EmaCrossoverLong)


cerebro.broker.setcash(10000.0)
cerebro.broker.setcommission(commission=0.001  )  # 0.1% commission


# Run the strategy and capture the strategy instance
results = cerebro.run()
strategy_instance = results[0]

# Access the enhanced trade pairs DataFrame
trade_pairs_df = strategy_instance.trade_pairs
trade_pairs_df.set_index('trade_number', inplace=True)

print("\n📊 COMPLETE TRADE PAIRS:")
# print(trade_pairs_df)


# Remove timezone for CSV
# df.index = df.index.tz_convert(None)
# trade_pairs_df.to_csv("SOLUSDT_SHORT_OUTCOME_FIVE2025_15m.csv")

# print("=" * 50)
print(f"🏁 Final Portfolio Value: ${cerebro.broker.getvalue():.2f}")



from datetime import time

# Convert the string to a time object
filter_time = time(13, 0, 0)
filter_data = trade_pairs_df[trade_pairs_df['entry_time'] >= filter_time]

total_pnl = filter_data['pnl'].sum()
total_positive = (filter_data['pnl'] > 0).sum()
total_negative = (filter_data['pnl'] < 0).sum()

print("Total PnL:", total_pnl)
print("Total Positive Trades:", total_positive)
print("Total Negative Trades:", total_negative)

trade_pairs_df



📊 COMPLETE TRADE PAIRS:
🏁 Final Portfolio Value: $5633.51
Total PnL: -363.72015913158475
Total Positive Trades: 47
Total Negative Trades: 96


,entry_type,entry_price,entry_date,entry_time,exit_type,exit_price,exit_date,exit_time,size,pnl,pnl_percentage
trade_number,,,,,,,,,,,
1,BUY,190.43,2025-01-01,11:15:00,SELL,190.04,2025-01-01,14:45:00,49.887098,-19.455968,-0.204800
2,BUY,190.79,2025-01-01,15:00:00,SELL,189.62,2025-01-01,17:15:00,49.604179,-58.036890,-0.613240
3,BUY,191.79,2025-01-01,18:45:00,SELL,194.12,2025-01-01,20:45:00,48.959457,114.075535,1.214870
4,BUY,208.19,2025-01-02,16:30:00,SELL,205.82,2025-01-02,17:45:00,45.539394,-107.928363,-1.138383
5,BUY,207.66,2025-01-02,20:30:00,SELL,206.49,2025-01-02,21:15:00,45.075620,-52.738476,-0.563421
...,...,...,...,...,...,...,...,...,...,...,...
301,BUY,174.63,2025-05-27,06:15:00,SELL,173.51,2025-05-27,07:00:00,31.928670,-35.760111,-0.641356
302,BUY,174.42,2025-05-28,12:15:00,SELL,173.30,2025-05-28,13:45:00,31.709979,-35.515177,-0.642128
303,BUY,171.43,2025-05-28,22:00:00,SELL,170.54,2025-05-28,22:45:00,32.005118,-28.484555,-0.519162
